### Scrape user playtimes using Steam API

We will use the Steam API to get user data. We will focus on the playtimes but see what other interesting information we might find. Maybe achievements as an indicator.

In [2]:
import pandas as pd
import os
import requests as re
import time
from tqdm import tqdm
import random

In [3]:
# import path, api key and steam id
USER_PLAYTIME_PATH = '../data/steam_playtime.csv'
STEAM_API_KEY = os.environ['STEAM_API']
MY_STEAM_ID = os.environ['STEAM_ID']

In [4]:
# urls for api calls
owned_url = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
getfriendlist_url = 'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/'

In [5]:
# If you have saved the data previously then load it. if it doesn't exist create it.
if os.path.exists(USER_PLAYTIME_PATH):
    playtime_df = pd.read_csv(USER_PLAYTIME_PATH)
else:
    playtime_df = pd.DataFrame(columns = ['appid', 'steam_id', 'playtime_2weeks', 'playtime_forever'])


In [6]:
len(playtime_df)

2731661

### Checking the API return
I know we can get playtime but what else

In [7]:
def parse_playtime_json(steamid, json_data) -> list[dict]:
    """ Get game playtimes from the json data and return a list of dicts """
    
    user_playtimes = [] 
    for game in json_data['response']['games']:
        if game['playtime_forever'] > 0:
            appid = game['appid']
            playtime_forever = game['playtime_forever']
            if 'playtime_2weeks' in game:
                playtime_2weeks = game['playtime_2weeks']
            else:
                playtime_2weeks = 0
            user_playtimes.append({'appid': appid, 
                                    'steam_id': steamid, 
                                    'playtime_2weeks': playtime_2weeks, 
                                    'playtime_forever': playtime_forever})
    return user_playtimes

In [8]:
def get_user_playtimes(steamid):
    """Function to get steam users playtime data"""
    
    global playtime_df
    
    # Check if we parsed this one, i.e. steam_id is not in our dataframe steam_id column
    if steamid not in playtime_df['steam_id'].values:
        urlparams = {'key': STEAM_API_KEY, 'steamid': steamid, 'format': 'json'}
        url_request = re.get(owned_url, params=urlparams)
        time.sleep(1)
        
        # If status_code is not 200, return
        if url_request.status_code != 200:
            print(url_request.status_code, end='')
            return
        
        req_json = url_request.json()
        if 'games' in req_json['response']:
            print('p', end='')  # p for parsing
            
            # Parse req_json fields for games user has played
            user_playtimes = []
            for game in req_json['response']['games']:
                if game['playtime_forever'] > 0:
                    appid = game['appid']
                    playtime_forever = game['playtime_forever']
                    playtime_2weeks = game.get('playtime_2weeks', 0)
                    
                    # Append to user_playtimes list
                    user_playtimes.append({'appid': appid, 
                                           'steam_id': steamid, 
                                           'playtime_2weeks': playtime_2weeks, 
                                           'playtime_forever': playtime_forever})
            
            # Append user_playtimes to playtime_df dataframe
            playtime_df = playtime_df.append(user_playtimes, ignore_index=True)


In [9]:
def friends_games(steamid, depth = 0):
    """ function to get friends steam id's recursively, recursive depth set to 6 """
    if depth == 6:
        return
    getfriendparams = {
        'key': STEAM_API_KEY,
        'steamid': steamid, 
        'relationship': 'all', 
        'format': 'json'}
    url_request = re.get(getfriendlist_url, params=getfriendparams)
    time.sleep(1)
    if url_request.status_code != 200:
        print(url_request.status_code, end='')
    else:
        req_json = url_request.json()
        if 'friendslist' in req_json:
            for friend in req_json['friendslist']['friends']:
                steamid = friend['steamid']
                # get playtime data for this friend
                get_user_playtimes(steamid)
                # run friends_games on this friend
                friends_games(steamid, depth+1)
                time.sleep(2)
        else:
            return

In [ ]:
# test run, parse through my friends
friends_games(MY_STEAM_ID)

### GET PLAYTIME DATA FROM STEAMID.UK ID's

In [ ]:
def get_userids_from_steamid_uk() -> pd.DataFrame:
    """ 
    parse the csv files from https://steamid.uk/downloads/
    and return a dataframe with unique userids. 
    Download files from the site above and place them in your data folder.
    :return: dataframe with unique userids from steamid.uk
    """

    sources = ['../data/3digitURLhistory04_06_2019.csv',
    '../data/3digitURLhistory16_04_2019.csv',
    '../data/3digitURLhistory18_01_2020.csv',
    '../data/3digitURLhistory18_03_2019.csv']
    
    source_col0 = '../data/3digitURLhistory26_02_2019.csv'

    # create DataFrame with feature 'someid'
    history = pd.DataFrame(columns=['someid'])

    # read in the csv files
    for source in sources:
        history = history.append(pd.read_csv(source, header=None, usecols=[1], names=['someid']))

    history = history.append(pd.read_csv(source_col0, header=None, usecols=[0], names=['someid']))

    # return unique ids
    return history.drop_duplicates()

In [ ]:
# create DataFrame with feature 'someid'
history = get_userids_from_steamid_uk()

In [ ]:
history.to_csv('../data/steamid_uk_userids.csv', index=False)

In [ ]:
# loop through data and get user playtimes, takes less than a day :)
for someid in tqdm(history['someid']):
    get_user_playtimes(someid)

In [14]:
len(playtime_df)

2806582

### GET PLAYTIME DATA RECURSIVELY

In [10]:
# create a list of unique steam_id's from playtime
steamids = list(set(playtime_df['steam_id']))

In [13]:
# loop that picks random steam_id's from steamids and runs friends_games on it
for i in range(5):
    random_steamid = random.choice(steamids)
    friends_games(random_steamid)
    time.sleep(5)

401p401401401401401pppppppppppppppppppppppppppppppppppppppppppppppppppppppp401ppppppppppppppppppppppppppp401pppppppp401401pppppppppppppppp401ppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppp

KeyboardInterrupt: 

In [15]:
# save data to csv
playtime_df.to_csv(USER_PLAYTIME_PATH, index=False)